In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import torchvision.models as models

In [ ]:
from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor()
                                      ])

In [ ]:
valid_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor()
                                      ])

In [ ]:
train_data = datasets.ImageFolder(root = '/content/drive/My Drive/Retail Pulse ML Assignment Data/train',transform=train_transforms)

In [ ]:
fil1 = open('/content/drive/My Drive/Retail Pulse ML Assignment Data/images_family_train.txt', 'r') 
Lin1 = fil1.readlines() 

In [ ]:
c = list()

In [ ]:
l = 0
temp = Lin1[0][8:]
for i in range(len(Lin1)):
    if(temp == Lin1[i][8:]):
        c.append(l)
    else:
        temp = Lin1[i][8:]
        l=l+1
        c.append(l)

In [ ]:
for i in range(len(Lin1)):
    train_data.targets[i] = c[i]
print(len(train_data.targets))

3334


In [ ]:
print(len(train_data.targets))

3334


In [ ]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [ ]:
BATCH_SIZE = 8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = models.resnet18(pretrained=True).to(device)
    
 
    
model.fc = nn.Sequential(
               nn.Linear(512, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 100)).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,480,660 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut7-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 31m 50s
	Train Loss: 4.631 | Train Acc: 0.93%
	 Val. Loss: 4.608 |  Val. Acc: 0.30%
Epoch: 02 | Epoch Time: 0m 22s
	Train Loss: 4.607 | Train Acc: 0.73%
	 Val. Loss: 4.610 |  Val. Acc: 0.30%
Epoch: 03 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.73%
	 Val. Loss: 4.612 |  Val. Acc: 0.30%
Epoch: 04 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.83%
	 Val. Loss: 4.614 |  Val. Acc: 0.30%
Epoch: 05 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.90%
	 Val. Loss: 4.616 |  Val. Acc: 0.30%
Epoch: 06 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.93%
	 Val. Loss: 4.618 |  Val. Acc: 0.30%
Epoch: 07 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.73%
	 Val. Loss: 4.619 |  Val. Acc: 0.30%
Epoch: 08 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 0.90%
	 Val. Loss: 4.620 |  Val. Acc: 0.30%
Epoch: 09 | Epoch Time: 0m 22s
	Train Loss: 4.606 | Train Acc: 1.00%
	 Val. Loss: 4.622 |  Val. Acc: 0.30%
Epoch: 10 | Epoch Time: 0m 22s
	Trai